Welcome to my 'National Pride' project

Soon you will be asked to choose a country, then I will retrive some 'ynet' articles for you regarding the country.
Using web scrapping to get the Header and Subtitle for each article
Later we will collect some intelligence about the country, like the flag image and how the people in the country sing their national anthem

In [1]:
# some imports, settings for Pandas

from google import search
import pandas as pd
from bs4 import BeautifulSoup
import requests
from IPython.display import display, HTML, Image

pd.set_option('max_colwidth', 500)
pd.options.display.max_rows = 500

In [ ]:
# scan country number

countries = [str(line.rstrip('\n')) for line in open('countries_names_hebrew.txt', encoding = 'utf-8')]
data_countries = pd.DataFrame({'': countries})
display(data_countries)
print('Enter country number and press Enter')

query = ""
country = int(input())
try:
    query = "site:www.ynet.co.il \"" + data_countries[""].iloc[country] + "\""

except IndexError:
    print("country input number should be in bounds (by list shown)")
    exit(1)
    
print(data_countries[""].iloc[country])

,
0,אוגנדה
1,אוזבקיסטן
2,אוסטריה
3,אוסטרליה
4,אוקראינה
5,אורוגוואי
6,אזרבייג'ן
7,איחוד האמירויות הערביות
8,איטליה
9,איי בהאמה


Enter country number and press Enter


In [ ]:
# get articles to the country from ynet.co.il

"""  @max_articles - max wanted articles to show"""
max_articles = 10
articles_count = 0 
urls = []
headers = []
subtitles = []

""" @stop - determines how many results will come back from google search,
should be bigger than @max_articles so we won't miss any wanted reuslts"""
for url in search(query, stop = 30):
    if 'articles' in url and articles_count < max_articles:
        soup = BeautifulSoup(requests.get(url).text, 'html.parser')
        header = soup.find("div", "art_header_title")
        subtitle = soup.find("div", "art_header_sub_title")
        if header is not None and subtitle is not None:
            urls.append(url)
            headers.append(header.string)
            subtitles.append(subtitle.string)

            articles_count += 1
            
data_articles = pd.DataFrame({'URL': urls, 'Header': headers, "Subtitle": subtitles})

![](https://www.ynet.co.il/images/CENTRAL_1024_ynet_logo.png)

In [ ]:
def make_clickable(url):
    return '<a target="_blank" href="{}">{}</a>'.format(url, url)

data_articles.style.format({'URL': make_clickable})

In [ ]:
# get country flag image

query = "site:he.wikipedia.org/wiki/ \"" + data_countries[""].iloc[country] + "\""
flag = None
try:
    for url in search(query, stop=10):
        soup = BeautifulSoup(requests.get(url).text, 'html.parser')
        flag = soup.find("img", "thumbborder")
        if flag is not None:
            flag = flag.get('src')
            flag_url = "http://" + flag[2:]
            break

# handle exceptions if no search results / search results don't fit the flag we wish for
except StopIteration:
    print("We couldn't find flag image")
except AttributeError:
    print("We couldn't find flag image")

In [ ]:
# output the flag image (if found)

"""align outputs to the center of the display"""

display(HTML("""
<style>
.output {
    display: flex;
    align-items: center;
}
</style>
"""))

if flag is not None:
    display(Image(flag_url))

In [ ]:
# get country anthem

query = "site:he.wikipedia.org/wiki/ \"" + "המנון " + data_countries[""].iloc[country] + "\""

anthem = None
anthems = []
data_countries = pd.DataFrame({'': countries})

try:
    for url in search(query, stop=10):
        soup = BeautifulSoup(requests.get(url).text, 'html.parser')
        anthem = soup.find_all('div', attrs={'class': 'poem'})
        # check if anthem is empty list
        if anthem:
            for language in anthem:
                anthem_data = pd.DataFrame({'': [line.strip() for line in language.text.split('\n')]})
                display(HTML(anthem_data.to_html(header = False)))
            break

    if not anthem:
        print("We couldn't find anthem lyrics")

# handle StopIteration for url assignment in try block above
except StopIteration:
    print("We couldn't find anthem lyrics")